In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [ ]:
shipment = pd.read_excel(path, sheet_name="出貨明細")

In [ ]:
shipment = shipment[(shipment["BG"] == "RF") & (shipment["狀態"].str.contains("出") & (shipment["預交年份"] == 2021) & (shipment["Group"] == "TELTONIKA"))]

In [ ]:
shipment = shipment[["Group", "預交月份", "品名", "單價", "數量"]]

In [ ]:
shipment["金額"] = shipment["單價"] * shipment["數量"]

In [ ]:
month_value = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_key = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
month_dict = dict(zip(month_key, month_value))

In [ ]:
price_list = shipment.drop_duplicates(subset=["品名", "單價"], keep="last")
product = price_list["品名"].tolist()
price = price_list["單價"].tolist()

In [ ]:
price_dict = dict(zip(product, price))

In [ ]:
shipment["類型"] = "實績"

In [ ]:
df = pd.read_excel(path, None)

In [ ]:
origin_2021 = df["2021 Forecast"]
revised_2021 = df["20210224update Q2"]

In [ ]:
origin_2021 = origin_2021[origin_2021["INPAQ PN"].isin(['DAM-P11-B-R2-015-10-49',
       'DAM-P11-B-R2-015-10-50', 'DAM-P11-E1-R2-003-22-03',
       'GPSGLONASSU10N-N6-0022-A', 'GPSGLONASSU10N-N6-0420-A',
       'PA1590MI4G-371-28M', 'PA1590MI4G-375-28M', 'PA1590MI4G-376-28M',
       'PA1590MI4G-377-28M', 'PA1590MI4G-402-28M', 'PA1590MI4G-447-28M-A',
       'PA1590MI4G-448-28M-A', 'PA1590MJ4G-221-26M', 'PA1590MJ4G-224-2SP',
       'PA1590MS4G-103-SMD-M-A', 'RFDPA141300SBLB803J4',
       'RFDPA171300SBAB823J4', 'RFDPA191300SMTB806J4'])]

In [ ]:
origin_2021 = origin_2021[['INPAQ PN', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12']]

In [ ]:
origin_2021 = origin_2021.groupby("INPAQ PN").sum().stack().reset_index()

In [ ]:
origin_2021.columns = ["品名", "預交月份", "數量"]

In [ ]:
origin_2021["預交月份"] = origin_2021["預交月份"].map(month_dict)

In [ ]:
origin_2021["單價"] = origin_2021["品名"].map(price_dict)

In [ ]:
origin_2021["金額"] = origin_2021["數量"] * origin_2021["單價"]

In [ ]:
origin_2021["類型"] = "初版預估"

In [ ]:
origin_2021["Group"] = "TELTONIKA"

In [ ]:
revised_2021 = revised_2021[revised_2021["INPAQ PN"].isin(['DAM-P11-B-R2-015-10-49',
       'DAM-P11-B-R2-015-10-50', 'DAM-P11-E1-R2-003-22-03',
       'GPSGLONASSU10N-N6-0022-A', 'GPSGLONASSU10N-N6-0420-A',
       'PA1590MI4G-371-28M', 'PA1590MI4G-375-28M', 'PA1590MI4G-376-28M',
       'PA1590MI4G-377-28M', 'PA1590MI4G-402-28M', 'PA1590MI4G-447-28M-A',
       'PA1590MI4G-448-28M-A', 'PA1590MJ4G-221-26M', 'PA1590MJ4G-224-2SP',
       'PA1590MS4G-103-SMD-M-A', 'RFDPA141300SBLB803J4',
       'RFDPA171300SBAB823J4', 'RFDPA191300SMTB806J4',
       'RFDPA30871ASBLBA01J4'])]

In [ ]:
revised_2021.columns = ['INPAQ PN', 'January', 'February', 'March', 'April',
       'May', 'June', 'July', 'August', 'September',
       'October', 'November', 'December']

In [ ]:
revised_2021 = revised_2021.groupby("INPAQ PN").sum().unstack().reset_index()

In [ ]:
revised_2021.columns = ['預交月份', 'INPAQ PN', "數量"]

In [ ]:
revised_2021["數量"] = revised_2021["數量"].astype("int")

In [ ]:
revised_2021 = revised_2021[revised_2021["數量"] != 0]

In [ ]:
revised_2021["單價"] = revised_2021["INPAQ PN"].map(price_dict)

In [ ]:
revised_2021["金額"] = revised_2021["單價"] * revised_2021["數量"]

In [ ]:
revised_2021["類型"] = "Q2修改"
revised_2021["Group"] = "TELTONIKA"

In [ ]:
origin_2021["數量"] = origin_2021["數量"].astype("int")

In [ ]:
shipment.columns

In [ ]:
origin_2021 = origin_2021.reindex(columns=shipment.columns)

In [ ]:
revised_2021.columns = ['預交月份', '品名', '數量', '單價', '金額', '類型', 'Group']

In [ ]:
revised_2021 = revised_2021.reindex(columns=shipment.columns)

In [ ]:
result = pd.concat([shipment, origin_2021, revised_2021], axis=0)

In [ ]:
plot_use = result.groupby(["預交月份", "品名", "類型"])[["金額"]].sum().reset_index()

In [ ]:
import plotly.express as px

In [ ]:
final_plot = px.bar(plot_use, x="預交月份", y="金額", color="類型", barmode="group", category_orders={"預交月份":month_value},
      title="TELTONIKA 2021年實績/預算/修改預算", labels={"金額":"總金額", "預交月份":"月份"}, text="品名").update_layout(title_font_size=26).update_xaxes(title_font={"size":18}).update_yaxes(title_font={"size":18})

In [ ]:
with open("teltonika預算實績差異.html", "a", encoding="utf-8") as f:
    f.write(final_plot.to_html(full_html=False, include_plotlyjs="cdn"))
    f.close()